In [3]:
import pandas as pd

# -------------------------
# CONFIGURATION & CONSTANTS
# -------------------------
TURKISH_MONTHS = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# -------------------------
# FUNCTIONS
# -------------------------

def load_index_data(filepath):
    df = pd.read_excel(filepath)
    df.columns = df.iloc[3][:3].to_list() + df.iloc[1][3:].to_list()
    df = df.iloc[4:]
    df = df.drop(columns=df.columns[df.columns.isna()])
    df = df.melt(id_vars=["Year", "Months"], var_name="inflation_code", value_name="index")
    df = df[(df['inflation_code'].str.len() == 5) | (df['inflation_code'].str.len() == 1)]
    return df

def load_weights(filepath):
    return pd.read_excel(filepath, dtype={"inflation_code": str})

def preprocess_data(df, weights):
    df = df.merge(weights, on=["inflation_code", "Year"], how="left")
    df = df[df.Year > 2014].fillna(100)
    df['Month_Num'] = df['Months'].map(TURKISH_MONTHS)
    df.sort_values(by=['inflation_code', 'Year', 'Month_Num'], inplace=True)
    return df

def calculate_cumulative_inflation(df):
    dec_df = df[df['Month_Num'] == 12][['Year', 'inflation_code', 'index']].copy()
    dec_df['Year'] += 1
    dec_df.rename(columns={'index': 'Prev_December_Index'}, inplace=True)
    df = df.merge(dec_df, on=['inflation_code', 'Year'], how='left')
    df['Cumulative_Inflation_%'] = ((df['index'] / df['Prev_December_Index']) - 1) * 100
    return df.dropna(subset=['Cumulative_Inflation_%'])

def compute_contributions(df):
    df['total_contribution'] = df['Cumulative_Inflation_%'] * df['weights'] / 100
    df['monthly_inf'] = df['index'].pct_change() * 100
    df.sort_values(by=['inflation_code', 'Year', 'Month_Num'], inplace=True)
    df['monthly_contribution_diff'] = df.groupby('inflation_code')['total_contribution'].diff()
    return df

def compute_correction(df):
    main_inf = df[df['inflation_code'] == '0'][['Year', 'Month_Num', 'monthly_inf']].rename(
        columns={'monthly_inf': 'main_monthly_inf'}
    )

    subgroup_df = df[df['inflation_code'] != '0']
    subgroup_sum = subgroup_df.groupby(['Year', 'Month_Num'])['monthly_contribution_diff'].sum().reset_index()
    subgroup_sum.rename(columns={'monthly_contribution_diff': 'total_contrib_diff'}, inplace=True)

    correction = pd.merge(main_inf, subgroup_sum, on=['Year', 'Month_Num'])
    correction['correction_coef'] = correction['main_monthly_inf'] / correction['total_contrib_diff']
    return correction[['Year', 'Month_Num', 'correction_coef']]

def apply_correction(df, correction):
    df = df.merge(correction, on=["Year", "Month_Num"])
    df['monthly_contribution'] = df['monthly_contribution_diff'] * df['correction_coef']
    return df

# -------------------------
# MAIN EXECUTION PIPELINE
# -------------------------
def process_inflation_data(index_path, weights_path, output_path="monthly_contribution.xlsx"):
    df = load_index_data(index_path)
    weights = load_weights(weights_path)
    df = preprocess_data(df, weights)
    df = calculate_cumulative_inflation(df)
    df = compute_contributions(df)
    correction = compute_correction(df)
    df = apply_correction(df, correction)

    # Select and sort final output
    output_columns = [
        'Year', 'Months', 'Month_Num', 'inflation_code', 'index',
        'Prev_December_Index', 'Cumulative_Inflation_%', 'weights',
        'total_contribution', 'monthly_inf', 'monthly_contribution_diff',
        'correction_coef', 'monthly_contribution'
    ]
    df = df[output_columns].dropna()
    df.sort_values(by=['Year', 'Month_Num', 'inflation_code'], inplace=True)
    
    df.to_excel(output_path, index=False)
    return df

# -------------------------
# RUN THE PIPELINE
# -------------------------
process_inflation_data(
        index_path="raw_data/harcama gruplarina gore endeks sonuclari.xlsx",
        weights_path="raw_data/cpi_item_weights_subgroups_5.xlsx",
        output_path="monthly_contribution.xlsx"
    )
final_df


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (__init__.py, line 1187)

In [4]:
pip install --upgrade xlrd

  Attempting uninstall: xlrd
    Found existing installation: xlrd 0.7.1
    Uninstalling xlrd-0.7.1:
      Successfully uninstalled xlrd-0.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camelot 12.6.29 requires xlrd==0.7.1, but you have xlrd 2.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
